In [20]:
from src.data_ingestion.document_loader import Document_Loader
doc = Document_Loader()
text = doc.document_loader("D:\\0MLOps\\RAG_Chatbot\\RAG_CHATBOT\\data\\text.txt")
text

"Greeting everyone. Today, I am here to deliver a speech on APJ Abdul Kalam. Dr APJ Abdul Kalam’s full name was Avul Pakir Zainuldeben Abdul Kalam, very few people know him by his full name as he was mostly addressed as ‘Missile Man of India’ and ‘People’s President’. He was born into a very poor family in Rameswaram on October 15, 1931. \n\n \n\nSince childhood, he enjoyed flying, and was equally curious to know how birds fly in the air? He was very intelligent and enjoyed reading, but his family did not have sufficient income for his school fees, so to support his education, he would wake up early in the morning and ride a bicycle 3 kilometres from home to collect newspapers and sell them. \n\n\nHe was admitted to St. Joseph's College, Tiruchirapalli, and later he went on to complete a degree in physics in 1954 and then studied at the Madras Institute of Technology and graduated in aeronautical engineering in 1955. Since his childhood, Dr Abdul Alam wanted to be a pilot but couldn’t 

In [21]:
from src.chunking.chunking import Chunking
chunking = Chunking(200,30)
chunks = chunking.sliding_window_chunking(text)
chunks

["Greeting everyone. Today, I am here to deliver a speech on APJ Abdul Kalam. Dr APJ Abdul Kalam’s full name was Avul Pakir Zainuldeben Abdul Kalam, very few people know him by his full name as he was mostly addressed as ‘Missile Man of India’ and ‘People’s President’. He was born into a very poor family in Rameswaram on October 15, 1931. Since childhood, he enjoyed flying, and was equally curious to know how birds fly in the air? He was very intelligent and enjoyed reading, but his family did not have sufficient income for his school fees, so to support his education, he would wake up early in the morning and ride a bicycle 3 kilometres from home to collect newspapers and sell them. He was admitted to St. Joseph's College, Tiruchirapalli, and later he went on to complete a degree in physics in 1954 and then studied at the Madras Institute of Technology and graduated in aeronautical engineering in 1955. Since his childhood, Dr Abdul Alam wanted to be a pilot but couldn’t make his dream

In [22]:
import chromadb
from chromadb.utils import embedding_functions


client = chromadb.PersistentClient(path="chroma_db")
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="all-MiniLM-L6-v2"
)

collection = client.get_or_create_collection(
    name="documents_collection",
    embedding_function=sentence_transformer_ef
)

In [23]:
def process_document(chunks,filename):
    try:
        chunks = chunks

        # Prepare metadata
        file_name = filename
        metadatas = [{"source": file_name, "chunk": i} for i in range(len(chunks))]
        ids = [f"{file_name}_chunk_{i}" for i in range(len(chunks))]

        return ids, chunks, metadatas
    except Exception as e:
        print(f"Error processing {filename}: {str(e)}")
        return [], [], []
a,b,c = process_document(chunks,"text.txt")

In [ ]:
def add_to_collection(collection, ids, texts, metadatas):
    if not texts:
        return

    batch_size = 200
    for i in range(0, len(texts), batch_size):
        end_idx = min(i + batch_size, len(texts))
        collection.add(
            documents=texts[i:end_idx],
            metadatas=metadatas[i:end_idx],
            ids=ids[i:end_idx]
        )

def process_and_add_documents(collection,chunks, filename: str):

    ids, texts, metadatas = process_document(chunks,filename)
    add_to_collection(collection, ids, texts, metadatas)
    print(f"Added {len(texts)} chunks to collection")
process_and_add_documents(collection,chunks, "text.txt")


Batches: 100%|██████████| 1/1 [00:00<00:00,  2.41it/s]

Added 5 chunks to collection


In [70]:
def semantic_search(collection, query: str, file_name:str, n_results: int = 2):

    results = collection.query(
        query_texts=[query],
        n_results=n_results
    )
    if file_name:
        filtered_text = []
        for i in range(n_results):
            if results['metadatas'][0][i]["source"] ==file_name:
                filtered_text.append({"ids": results['ids'][0][i],
                                        "documents": results["documents"][0][i],
                                        "metadatas": results['metadatas'][0][i],
                                        "distances": results['distances'][0][i]
                                        })
            
    return filtered_text
query = "When did Dr. APJ Abdul Kalam pass away, and how?"
results = semantic_search(collection,query, file_name="text.txt",n_results=2)
results

Batches: 100%|██████████| 1/1 [00:00<00:00, 31.32it/s]


[{'ids': 'text.txt_chunk_2',
  'documents': "awarded an honorary doctorate by more than 30 universities in the world for the same. In 2002, he was elected President of India and was the country's first scientist and non-political president. He visited many countries during his tenure as President and led India's youth through his lectures and encouraged them to move forward. ‘My vision for India’ was a Famous Speech of APJ Abdul Kalam delivered at IIT Hyderabad in 2011, and is to this day my favourite speech. His far-reaching thinking gave India's growth a fresh path and became the youth's inspiration. Dr Abdul Kalam died on July 27, 2015, from an apparent cardiac arrest while delivering a lecture at IIM Shillong at the age of 83. He spent his entire life in service and inspiration for the nation and the youth, and his death is also while addressing the youth. His death is a never-ending loss to the country. Short APJ Abdul Kalam Speech In English For Students Today, I am here to deliv

In [93]:
def formated_context_with_sources(results):
    context = results[0]['documents']
    for i in range(1,len(results)):
        context += "\n\n"+ results[i]["documents"]
    sources = [f"{i['metadatas']['source']}(chunk {i['metadatas']['chunk']})" 
               for i in results]

    return context, sources


In [94]:
formated_context_with_sources(results)

("awarded an honorary doctorate by more than 30 universities in the world for the same. In 2002, he was elected President of India and was the country's first scientist and non-political president. He visited many countries during his tenure as President and led India's youth through his lectures and encouraged them to move forward. ‘My vision for India’ was a Famous Speech of APJ Abdul Kalam delivered at IIT Hyderabad in 2011, and is to this day my favourite speech. His far-reaching thinking gave India's growth a fresh path and became the youth's inspiration. Dr Abdul Kalam died on July 27, 2015, from an apparent cardiac arrest while delivering a lecture at IIM Shillong at the age of 83. He spent his entire life in service and inspiration for the nation and the youth, and his death is also while addressing the youth. His death is a never-ending loss to the country. Short APJ Abdul Kalam Speech In English For Students Today, I am here to deliver a speech on Dr APJ Abdul Kalam. APJ Abdu